## Model Fiting and Prediction

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt 
import xgboost as xgb
from sklearn.metrics import auc
from sklearn.model_selection import GridSearchCV
import pickle

In [4]:
train = pd.read_csv('input/train_fe.csv')

In [5]:
X  = train.drop(columns = ['Unnamed: 0', 'is_attributed'])
y  = train['is_attributed'].astype('uint8')
del train

In [17]:
param_set = {'n_estimators':[30,40,50],
             'learning_rate': [0.01,0.05,0.1,1],
             'num_leaves': [3,5,7,9],  # we should let it be smaller than 2^(max_depth)
             'max_depth': [4,5,7,8],  # -1 means no limit
             'max_bin': [50,100,200],  # Number of bucketed bin for feature values
             }

estimator = lgb.LGBMClassifier(boosting_type='gbdt', 
                           objective='binary',
                           metric='auc',
                           min_split_gain=0,
                           min_child_weight=5,
                           min_child_samples=10,
                           subsample= 0.7,  # Subsample ratio of the training instance.
                           subsample_freq=1,  # frequence of subsample, <=0 means no enable\
                           colsample_bytree=0.7,  # Subsample ratio of columns when constructing each tree.
                           reg_alpha=1, 
                           reg_lambda=0,
                           seed=410, 
                           nthread=4, 
                           silent=True)


gsearch = GridSearchCV(estimator, param_grid = param_set, scoring='roc_auc',n_jobs=1,iid=False, cv=10)

lgb_model = gsearch.fit(X.values, y.values)
lgb_model.best_params_, lgb_model.best_score_

/home/jacob/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.95685, std: 0.01548, params: {'learning_rate': 0.01, 'max_bin': 50, 'max_depth': 4, 'n_estimators': 30, 'num_leaves': 3},
  mean: 0.96505, std: 0.01327, params: {'learning_rate': 0.01, 'max_bin': 50, 'max_depth': 4, 'n_estimators': 30, 'num_leaves': 5},
  mean: 0.96856, std: 0.01129, params: {'learning_rate': 0.01, 'max_bin': 50, 'max_depth': 4, 'n_estimators': 30, 'num_leaves': 7},
  mean: 0.97043, std: 0.01097, params: {'learning_rate': 0.01, 'max_bin': 50, 'max_depth': 4, 'n_estimators': 30, 'num_leaves': 9},
  mean: 0.95752, std: 0.01592, params: {'learning_rate': 0.01, 'max_bin': 50, 'max_depth': 4, 'n_estimators': 40, 'num_leaves': 3},
  mean: 0.96521, std: 0.01369, params: {'learning_rate': 0.01, 'max_bin': 50, 'max_depth': 4, 'n_estimators': 40, 'num_leaves': 5},
  mean: 0.96912, std: 0.01116, params: {'learning_rate': 0.01, 'max_bin': 50, 'max_depth': 4, 'n_estimators': 40, 'num_leaves': 7},
  mean: 0.97098, std: 0.01076, params: {'learning_rate': 0.01, 'max_bin': 50

In [ ]:
lgb_params = {'boosting_type': 'gbdt',
              'objective': 'binary',
              'metric':'auc',
              'learning_rate': 0.05,
              'num_leaves': 9,
              'max_depth': 7,
              'min_child_samples': 10,
              'max_bin': 100,
              'subsample': 0.7,
              'subsample_freq': 1,
              'colsample_bytree': 0.7,
              'min_child_weight': 5,
              'min_split_gain': 0,
              'reg_alpha': 1,
              'reg_lambda': 0,
              'nthread': 4,
              'verbose': 0,
             }

predictors = list(X.columns())
categorical_features = []
xgtrain = lgb.Dataset(dtrain[predictors].values, label=y.values,
                      feature_name=predictors,
                      categorical_feature=categorical_features
                     )

evals_results = {}

lightGBM_model = lgb.train(lgb_params, 
                     xgtrain, 
                     evals_result=evals_results, 
                     verbose_eval=10, 
                     feval=feval)

In [19]:
filename = 'lgb_model.sav'
pickle.dump(lgb_model, open(filename, 'wb'))
# some time later.. load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))

### Use lightGBM predict each test_fe file and merge into one CSV

In [47]:
for i in np.arange(127)+1:
    print('Begin to forecast for file: ' + 'input/test_fe/test_'+str(i)+'fe.csv')
    test = pd.read_csv('input/test_fe/test_'+str(i)+'fe.csv', header=0)
    test = test.drop(columns=['Unnamed: 0'])
    if i == 1:
        p = lgb_model.predict(test.values)
    else:
        p = np.append(p,lgb_model.predict(test.values))
    print('forecast finished: '+ str(len(p))+'. Total Work finished: '+ str(i)+ '/127')

Begin to forecast for file: input/test_fe/test_1fe.csv


/home/jacob/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


forecast finished: 148000. Total Work finished: 1/127
Begin to forecast for file: input/test_fe/test_2fe.csv


/home/jacob/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


forecast finished: 296000. Total Work finished: 2/127
Begin to forecast for file: input/test_fe/test_3fe.csv


...

/home/jacob/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


forecast finished: 18500000. Total Work finished: 125/127
Begin to forecast for file: input/test_fe/test_126fe.csv


/home/jacob/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


forecast finished: 18648000. Total Work finished: 126/127
Begin to forecast for file: input/test_fe/test_127fe.csv
forecast finished: 18790469. Total Work finished: 127/127


/home/jacob/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [51]:
sub = pd.DataFrame()
sub.loc[:,'click_id'] = np.arange(len(p))
sub.loc[:,'is_attributed'] = p
sub.to_csv('/output/ligGBM_forecast.csv',index=False)

### Using Xgboost

In [52]:
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

/home/jacob/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/jacob/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [56]:
#test_results = pd.read_csv('test_results.csv')
def modelfit(alg, X, y,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):

    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X.values, label=y.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
             early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #建模
    alg.fit(X.values ,y.values ,eval_metric='auc')
        
    #对训练集预测
    dtrain_predictions = alg.predict(X.values)
    dtrain_predprob = alg.predict_proba(X.values)[:,1]
        
    #输出模型的一些结果
    print("准确率 : %.4g" % metrics.accuracy_score(y.values, dtrain_predictions))
    print("AUC 得分 (训练集): %f" % metrics.roc_auc_score(y, dtrain_predprob))

In [57]:
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
modelfit(xgb1, X, y)

/home/jacob/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


准确率 : 0.9558
AUC 得分 (训练集): 0.993632


In [ ]:
# 对于max_depth和min_child_weight查找最好的参数
param_test1 = {
    'max_depth':np.arange(3,10,2),
    'min_child_weight':np.arange(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X,y)
gsearch1.best_params_, gsearch1.best_score_

In [ ]:
# 对于max_depth和min_child_weight查找最好的参数
param_test2 = {
    'max_depth':[4,5,6],
    'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X,y)
gsearch2.best_params_, gsearch2.best_score_

In [ ]:
#交叉验证对min_child_weight寻找最合适的参数
param_test2b = {
    'min_child_weight':[6,8,10,12]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=4,
                                        min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2b.fit(X,y)
gsearch2b.best_params_, gsearch2b.best_score_

In [ ]:
#Grid seach选择合适的gamma
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=4,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X,y)
gsearch3.best_params_, gsearch3.best_score_

In [ ]:
#对subsample 和 colsample_bytree用grid search寻找最合适的参数
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(train[predictors],train[target])
gsearch4.best_params_, gsearch4.best_score_

In [ ]:
# 同上
param_test5 = {
    'subsample':[i/100.0 for i in range(75,90,5)],
    'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(train[predictors],train[target])
gsearch5.best_params_, gsearch5.best_score_

In [ ]:
#对reg_alpha用grid search寻找最合适的参数
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
                                        min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(train[predictors],train[target])
gsearch6.best_params_, gsearch6.best_score_

In [ ]:
# 换一组参数对reg_alpha用grid search寻找最合适的参数
param_test7 = {
    'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch7 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
                                        min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch7.fit(train[predictors],train[target])
gsearch7.best_params_, gsearch7.best_score_

### Blend prediction from different models

In [ ]:
# All credits go to original authors!
import pandas as pd

test_files = ['../input/lewis-undersampler-9562-version/pred.csv',
              '../input/weighted-app-chanel-os/subnew.csv',
              '../input/single-xgboost-lb-0-9639/xgb_sub.csv',
              '../input/lightgbm-with-count-features/sub_lgb_balanced99.csv'
              ]

model_test_data = []
for test_file in test_files:
    print('read ' + test_file)
    model_test_data.append(pd.read_csv(test_file, encoding='utf-8'))
n_models = len(model_test_data)

weights = [0.10, 0.15, 0.25, 0.50]
column_name = 'is_attributed'

print('predict')
test_predict_column = [0.] * len(model_test_data[0][column_name])
for ind in range(0, n_models):
    test_predict_column += model_test_data[ind][column_name] * weights[ind]

print('make result')
final_result = model_test_data[0]['click_id']
final_result = pd.concat((final_result, pd.DataFrame(
    {column_name: test_predict_column})), axis=1)
final_result.to_csv("average_result.csv", index=False)